![](2022-10-18-13-26-37.png)

In [63]:
import pyomo.environ as pyo 
from pyomo.opt import SolverFactory


In [64]:
model = pyo.ConcreteModel()


In [65]:
## parameters
model.T = pyo.Param(initialize = 10)
T = model.T
model.M = pyo.Param(initialize = 4)
M = model.M

model.limProd = pyo.Param(initialize = 10)

limProd = model.limProd 

## set

model.setT = pyo.Set(initialize = range(1,T+1))

setT = model.setT

In [66]:
from tkinter.tix import INTEGER


model.x = pyo.Var(range(1,M+1), setT,within = pyo.Integers, bounds = (0,10))
x = model.x
model.pprint()

3 Set Declarations
    setT : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain         : Size : Members
        None :     2 : x_index_0*setT :   40 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}

3 Param Declarations
    M : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     4
    T : Size=1, Index=None, Domain=Any, Default=None, M

In [67]:
# Objective

model.obj = pyo.Objective(expr = sum([x[m,t] for m in range (1,M+1) for t in range(1,T+1)]), sense = pyo.maximize)


In [68]:
# Constraint

# model.C1 = pyo.ConstraintList() 

# for t in range(1, T+1):
#     model.C1.add(expr = 2*x[2,t]- 8*x[3,t]<= 0)

def rule1(model , t):
    return 2*model.x[2,t] - 8* model.x[3,t] <=0

model.C1 = pyo.Constraint(model.setT, rule=rule1)  ## Sample rule in place of constraint list

def rule2(model,t):
    return model.x[2,t] - 2*model.x[3,t-2] + model.x[4,t] >=1

model.C2 = pyo.Constraint(range(3, T+1), rule = rule2)   
 
# model.C2 = pyo.ConstraintList()

# for t in range(3, T+1):
#     model.C2.add(expr = x[2,t] - 2*x[3,t-2] +x[4,t] >=1)

model.C3 = pyo.ConstraintList()

for t in range(1,T+1):
    model.C3.add(expr = sum([x[m,t] for m in range (1,M+1)])<=50)

model.C4 = pyo.ConstraintList()

for t in range(2, T+1):
    model.C4.add(expr = x[1,t]+x[2,t-1]+x[3,t]+x[4,t]<=limProd)



In [69]:
opt = SolverFactory('gurobi')

opt.options['MIPgap'] = 0.01 # Gap limit

opt.options['TimeLimit'] = 60  # Time Limit

results = opt.solve(model, tee=True)

Restricted license - for non-production use only - expires 2023-10-25
Read LP format model from file /tmp/tmpb_dbubrz.pyomo.lp
Reading time = 0.00 seconds
x41: 38 rows, 41 columns, 121 nonzeros
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 38 rows, 41 columns and 121 nonzeros
Model fingerprint: 0x8e3e7e06
Variable types: 1 continuous, 40 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 110.0000000
Presolve removed 18 rows and 20 columns
Presolve time: 0.00s
Presolved: 20 rows, 21 columns, 55 nonzeros
Found heuristic solution: objective 124.0000000
Variable types: 0 continuous, 21 integer (0 binary)

Root relaxation: objective 1.280000e+02, 12 ite

In [70]:
model.pprint()

6 Set Declarations
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {3, 4, 5, 6, 7, 8, 9, 10}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    setT : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain         : Size : Members
        None :     2 : x_index_0*setT :   40 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10),